## BACKTESTING OF Signals

In [1]:
import sys
import time
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY

import mysql.connector
from mysql.connector import errorcode

print(sys.version)

3.5.4 |Anaconda custom (x86_64)| (default, Sep 21 2017, 18:03:17) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


## connect to DB

In [2]:
DATABASE = 'prodcopy'

if DATABASE == 'prod':
    config = {
      'user': 'alienbaby',
      'password': 'alienbabymoonangel',
      'host': 'intelligenttrading-aurora-production-primary-cluster.cluster-caexel1tmds5.us-east-1.rds.amazonaws.com',
      'port': '3306',
      'database': 'intelligenttrading_primary',
      'raise_on_warnings': True,
    }
    try:
      db_connection = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
      if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
      elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
      else:
        print(err)
        
elif DATABASE == 'stage':
    config = {
      'user': 'alienbaby',
      'password': 'alienbabymoonangel',
      'host': 'intelligenttrading-aurora-production-postgres-cluster.cluster-caexel1tmds5.us-east-1.rds.amazonaws.com',
      'port': '5432',
      'dbname': 'primary_postgres'
    }

    try:
      db_connection = pg.connect(**config)
    except mysql.connector.Error as err:
      if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
      elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
      else:
        print(err)
        
elif DATABASE == 'prodcopy':
    config = {
      'user': 'alienbaby',
      'password': 'alienbabymoonangel',
      'host': 'prodclone.caexel1tmds5.us-east-1.rds.amazonaws.com',
      'port': '3306',
      'database': 'intelligenttrading_primary',
      'raise_on_warnings': True,
    }

    try:
      db_connection = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
      if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
      elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
      else:
        print(err)

## Get past signals and prices from staging DB


In [68]:
transaction_coin = 'OMG'
counter_coin = 0
resample_period = 60

sgn = 'kumo_breakout'

### get prices

In [69]:
query = (" SELECT * FROM indicator_priceresampl WHERE \
transaction_currency='%s' AND \
counter_currency=%d \
and resample_period=%d " ) % (transaction_coin, counter_coin, resample_period)

resampl_df = pd.read_sql(query, con=db_connection)

# make index a timestamp relative_strength
resampl_df['timestamp'] = pd.to_datetime(resampl_df['timestamp'], unit='s')
resampl_df.index = pd.DatetimeIndex(resampl_df.timestamp)


resampl_df.sort_index(inplace=True)

In [70]:
resampl_df.tail(3)

,id,source,counter_currency,transaction_currency,timestamp,resample_period,open_price,close_price,low_price,high_price,midpoint_price,mean_price,price_variance
timestamp,,,,,,,,,,,,,
2018-02-27 08:00:00,104591,0,0,OMG,2018-02-27 08:00:00,60,176780.0,175615.0,175329.0,176780.0,176054.0,175919.0,119592.298028
2018-02-27 09:00:00,104670,0,0,OMG,2018-02-27 09:00:00,60,176000.0,177689.0,176000.0,178000.0,177000.0,177629.0,178307.072884
2018-02-27 10:00:00,104749,0,0,OMG,2018-02-27 10:00:00,60,176260.0,177618.0,174776.0,177618.0,176197.0,175847.0,567293.797193


### get signals

In [71]:
query = " SELECT * FROM signal_signal WHERE \
resample_period=%d AND transaction_currency='%s' AND counter_currency=%d  " % (resample_period, transaction_coin, counter_coin)
#query = " SELECT * FROM signal_signal"
signals_df = pd.read_sql(query, con=db_connection)

# make index a timestamp relative_strength
signals_df['timestamp'] = pd.to_datetime(signals_df['timestamp'], unit='s')
signals_df.index = pd.DatetimeIndex(signals_df.timestamp)
signals_df.sort_index(inplace=True)

In [72]:
signals_df[signals_df.signal==sgn]

,id,created_at,modified_at,UI,subscribers_only,text,source,transaction_currency,signal,trend,...,price_change,volume_btc,volume_btc_change,volume_usdt,volume_usdt_change,sent_at,timestamp,rsi_value,counter_currency,resample_period
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-30 08:44:54.497020,69059,2018-01-30 09:07:23.481244,2018-01-30 09:07:23.579788,0,1,,0,OMG,kumo_breakout,1,...,0.091382,None,None,None,None,1.517303e+09,2018-01-30 08:44:54.497020,NaN,0,60
2018-02-01 15:34:16.166530,69875,2018-02-01 15:39:12.260134,2018-02-01 15:39:12.375331,0,1,,0,OMG,kumo_breakout,-1,...,-0.007732,None,None,None,None,1.517500e+09,2018-02-01 15:34:16.166530,NaN,0,60
2018-02-01 16:46:56.243280,69938,2018-02-01 16:51:29.198100,2018-02-01 16:51:29.337282,0,1,,0,OMG,kumo_breakout,-1,...,-0.019913,None,None,None,None,1.517504e+09,2018-02-01 16:46:56.243280,NaN,0,60
2018-02-02 09:16:31.355470,70317,2018-02-02 09:21:22.248269,2018-02-02 09:21:22.347000,0,1,,0,OMG,kumo_breakout,-1,...,-0.078947,None,None,None,None,1.517563e+09,2018-02-02 09:16:31.355470,NaN,0,60
2018-02-02 10:26:35.412800,70360,2018-02-02 10:32:00.519908,2018-02-02 10:32:00.604586,0,1,,0,OMG,kumo_breakout,-1,...,-0.053532,None,None,None,None,1.517568e+09,2018-02-02 10:26:35.412800,NaN,0,60
2018-02-03 05:08:14.808380,70882,2018-02-03 05:11:27.121334,2018-02-03 05:11:27.328361,0,1,,0,OMG,kumo_breakout,-1,...,0.000470,None,None,None,None,1.517635e+09,2018-02-03 05:08:14.808380,NaN,0,60
2018-02-03 06:15:43.527300,70890,2018-02-03 06:18:42.084667,2018-02-03 06:18:42.186821,0,1,,0,OMG,kumo_breakout,-1,...,0.019308,None,None,None,None,1.517639e+09,2018-02-03 06:15:43.527300,NaN,0,60
2018-02-05 05:44:44.418550,71450,2018-02-05 05:47:48.443007,2018-02-05 05:47:48.601713,0,1,,0,OMG,kumo_breakout,-1,...,-0.045173,None,None,None,None,1.517810e+09,2018-02-05 05:44:44.418550,NaN,0,60
2018-02-05 09:07:41.540320,71483,2018-02-05 09:11:31.119539,2018-02-05 09:11:31.233772,0,1,,0,OMG,kumo_breakout,-1,...,-0.086345,None,None,None,None,1.517822e+09,2018-02-05 09:07:41.540320,NaN,0,60


In [62]:
all_signals = signals_df[signals_df.signal==sgn]

cumulative_sum = 0
for _,row in all_signals.iterrows():
    tm = row.timestamp
    tr = row.trend
    print(resampl_df[resampl_df.timestamp==tm].mean_price)

timestamp
2018-02-01 15:34:16.166530   NaN
Name: mean_price, dtype: float64
timestamp
2018-02-01 16:46:56.243280    11487028.0
Name: mean_price, dtype: float64
timestamp
2018-02-01 17:59:09.600090    11639946.0
Name: mean_price, dtype: float64
timestamp
2018-02-02 15:27:20.457500    10582247.0
Name: mean_price, dtype: float64


In [57]:
all_signals

,id,created_at,modified_at,UI,subscribers_only,text,source,transaction_currency,signal,trend,...,price_change,volume_btc,volume_btc_change,volume_usdt,volume_usdt_change,sent_at,timestamp,rsi_value,counter_currency,resample_period
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-02-01 15:34:16.166530,69857,2018-02-01 15:36:02.035285,2018-02-01 15:36:02.132147,0,1,,0,ETH,kumo_breakout,-1,...,0.082107,None,None,None,None,1.517499e+09,2018-02-01 15:34:16.166530,NaN,0,60
2018-02-01 16:46:56.243280,69921,2018-02-01 16:48:27.918666,2018-02-01 16:48:28.018956,0,1,,0,ETH,kumo_breakout,1,...,0.045533,None,None,None,None,1.517504e+09,2018-02-01 16:46:56.243280,NaN,0,60
2018-02-01 17:59:09.600090,69983,2018-02-01 18:01:20.700559,2018-02-01 18:01:20.831920,0,1,,0,ETH,kumo_breakout,1,...,0.055812,None,None,None,None,1.517508e+09,2018-02-01 17:59:09.600090,NaN,0,60
2018-02-02 15:27:20.457500,70579,2018-02-02 15:28:47.335466,2018-02-02 15:28:47.501032,0,1,,0,ETH,kumo_breakout,1,...,-0.092154,None,None,None,None,1.517585e+09,2018-02-02 15:27:20.457500,NaN,0,60
